In [ ]:
!nvidia-smi

Fri Mar 21 08:17:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [ ]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [ ]:
!pip install sacrebleu -q
!pip install tqdm -q
!pip install numpy -q
!pip install huggingface_hub -q
!pip install faster-whisper -q
!pip install transformers -q
!pip install evaluate -q
!pip install jiwer -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset, Audio, DatasetDict, config
import pandas as pd
from tqdm import tqdm
from librosa import load, get_duration
from tqdm.notebook import tqdm
from transformers import WhisperFeatureExtractor, WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import torch
from torch.utils.data import IterableDataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

# Get user info
user_info = whoami()
print(user_info)

{'type': 'user', 'id': '6573e3d7244aefdfc476f07a', 'name': 'cobrayyxx', 'fullname': 'M Hazim AF', 'email': 'muhammadhazim86@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/16ef7a7d5351781180384c7f5b4ef392.svg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '67a769f44edabdd17b5c169b', 'name': 'SpeechT', 'fullname': 'SpeechT', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/618c42e4f35289d4bcab9cf1/m1k2sALFEwa9B2McXNyOi.png', 'roleInOrg': 'read', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'learning', 'role': 'w

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Download Dataset

In [ ]:
train_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025",split="train+val") # Todo: add train+val

README.md:   0%|          | 0.00/620 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
test_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025", split="test", streaming=True)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
train_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 41
})

In [ ]:
test_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 2
})

# Preprocessing Dataset

In [ ]:
train_dataset_cleaned = train_dataset.filter(lambda x: x["sentence"] is not None and x["translation"] is not None and x["audio"] is not None)

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
test_dataset_cleaned = test_dataset.filter(lambda x: x["sentence"] is not None and x["translation"] is not None and x["audio"] is not None)

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
train_dataset_cleaned

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

In [ ]:
test_dataset_cleaned

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

# Testing with first 10 data (this is just for checking whether the training code is working or not)

In [ ]:
# Get the first 10 examples
train_dataset = list(train_dataset.take(10))
test_dataset = list(test_dataset.take(10))

In [ ]:
# Print the first 10 examples
for i, sample in enumerate(train_dataset):
    print(f"Sample {i+1}: {sample}")

Sample 1: {'audio': {'path': '26_8819_0_261_01_211112-010515_bem_8f1_elicit_0.wav', 'array': array([0.        , 0.        , 0.        , ..., 0.        , 0.00012207,
       0.00018311]), 'sampling_rate': 16000}, 'sentence': 'Abalumendo babili nabeminina bale ikata amabula yafimuti.', 'translation': 'these men are standing caching tree leaves', 'speaker_id': 261}
Sample 2: {'audio': {'path': '38_11847_4_381_01_211114-161237_bem_ba5_elicit_2.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'sentence': 'Cifwile uyu ou bali nankwe eulemulanga ifyakucita.', 'translation': 'It must be that the person they are with is showing them what to do.', 'speaker_id': 381}
Sample 3: {'audio': {'path': '24_2645_0_241_01_211110-143805_bem_150_elicit_0.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00372314,
       -0.0027771 ,  0.00289917]), 'sampling_rate': 16000}, 'sentence': 'Umulumendo ekele pacipuna elo kumaboko ekete icilubi icakashikila.', 'transla

In [ ]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [ ]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

In [ ]:
test_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})



---



#Prepare Dataset

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
input_str = train_dataset_cleaned["sentence"][0]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Abalumendo babili nabeminina bale ikata amabula yafimuti.
Decoded w/ special:    <|startoftranscript|><|be|><|transcribe|><|notimestamps|>Abalumendo babili nabeminina bale ikata amabula yafimuti.<|endoftext|>
Decoded w/out special: Abalumendo babili nabeminina bale ikata amabula yafimuti.
Are equal:             True


In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="transcribe")

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    #compute log-Me1 input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"],
                                max_length=448,  # Ensure target length is within model limits
                                truncation=True,
                                padding="max_length").input_ids
    return batch


In [ ]:
train_dataset_cleaned.column_names

['audio', 'sentence', 'translation', 'speaker_id']

In [ ]:
train_converted = train_dataset_cleaned.map(prepare_dataset, remove_columns=train_dataset_cleaned.column_names, num_proc=1)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
train_converted

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 10
})

In [ ]:
test_converted = test_dataset_cleaned.map(prepare_dataset, remove_columns=test_dataset_cleaned.column_names, num_proc=1)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

# Prepare for Training

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "en"
model.generation_config.task = "transcribe"

# model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")
metric_wer = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    bleu = round(bleu["score"], 2)

    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"bleu": bleu, "chrf": chrf, "wer": wer}

# Training

In [ ]:
# adjust the variable below
hf_repo = "cobrayyxx"
output_dir = "whisper-small-be2en"

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # Change this to your desired output directory
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # Increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_ratio=0.03,
    num_train_epochs=3,
    gradient_checkpointing=True,

    bf16=True,
    hub_model_id=f"{hf_repo}/{output_dir}",
    # Changed to evaluate at the end of each epoch
    evaluation_strategy="epoch",

    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,

    # Save best model based on WER metric at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="chrf",
    greater_is_better=True,  # Lower WER is better, for CHRF greater is better

    # More frequent logging for better monitoring
    logging_steps=1,  # Adjust if needed
    report_to=["tensorboard"],

    push_to_hub=True,  # Upload model to Hugging Face Hub if needed
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_converted,
    eval_dataset=test_converted,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-43-f93e445828c7>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Wer
1,2.169700,2.158247,0.470000,0.260000,117.142857
2,2.169700,0.231016,0.000000,0.000000,100.000000
3,0.229000,0.161809,0.900000,34.000000,97.142857


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 

TrainOutput(global_step=3, training_loss=1.5227681746085484, metrics={'train_runtime': 173.9545, 'train_samples_per_second': 0.172, 'train_steps_per_second': 0.017, 'total_flos': 8657562009600000.0, 'train_loss': 1.5227681746085484, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1742545421.9be5e65dbdef.1583.0:   0%|          | 0.00/9.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cobrayyxx/whisper-small-be2en/commit/0992d4ce5a1e9676a339db546bf81e7d426fba22', commit_message='End of training', commit_description='', oid='0992d4ce5a1e9676a339db546bf81e7d426fba22', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cobrayyxx/whisper-small-be2en', endpoint='https://huggingface.co', repo_type='model', repo_id='cobrayyxx/whisper-small-be2en'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import WhisperTokenizerFast

# Load your fine-tuned tokenizer
tokenizer = WhisperTokenizerFast.from_pretrained("openai/whisper-small", language="en", task="transcribe")

# Save the tokenizer locally
tokenizer.save_pretrained(output_dir,legacy_format=False)

# Push the tokenizer to the Hugging Face Hub
tokenizer.push_to_hub(f"{hf_repo}/{output_dir}")

README.md:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cobrayyxx/whisper-small-be2en/commit/2576b558e5ad5d5598ef57da9de2bd73736a5280', commit_message='Upload tokenizer', commit_description='', oid='2576b558e5ad5d5598ef57da9de2bd73736a5280', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cobrayyxx/whisper-small-be2en', endpoint='https://huggingface.co', repo_type='model', repo_id='cobrayyxx/whisper-small-be2en'), pr_revision=None, pr_num=None)